In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import os
import time

In [ ]:
population = pd.read_csv("population.csv")

In [ ]:
"""
For when you want to get just the countries
population = population[population['Country Name'].str.find('income') == -1]
population = population[population['Country Name'].str.find('countries') == -1]
population = population[population['Country Name'].str.find('IDA') == -1]
population = population[population['Country Name'].str.find('BDA') == -1]
population = population[population['Country Name'].str.find('World') == -1]
population = population[population['Country Name'].str.find('demographic') == -1]
population = population[population['Country Name'].str.find('members') == -1]
population = population[population['Country Name'].str.find('situations') == -1]
population = population[population['Country Name'].str.find('&') == -1]
population = population[population['Country Name'].str.find('IBRD') == -1]
population = population[population['Country Name'].str.find('South Asia') == -1]
population = population[population['Country Name'].str.find('Union') == -1]
population = population[population['Country Name'].str.find('North America') == -1]
population = population[population['Country Name'].str.find('area') == -1]
population = population[population['Country Name'].str.find('Sub-Saharan Africa') == -1]
population = population[population['Country Name'].str.find('Central Europe and the Baltics') == -1]
"""

In [ ]:
population = population[population['Country Name'].str.find('income') != -1]
population = population[population['Country Name'].str.find('(') == -1]
population.head()

In [ ]:
population.drop(columns={'Unnamed: 64','2019','Indicator Name','Indicator Code','Country Code'},inplace=True)

In [ ]:
population.fillna(0,inplace=True)
population.head(10)
population.to_csv("income.csv")


In [ ]:
population = pd.read_csv("income.csv")

In [ ]:
for p in range(3):
    i = 0
    while i < len(population.columns):
        try:
            a = np.array(population.iloc[:, i + 1])
            b = np.array(population.iloc[:, i + 2])
            c = (a + b) / 2
            population.insert(i+2, str(population.iloc[:, i + 1].name) + '^' + str(len(population.columns)), c)
        except:
            print(f"\n  Interpolation No. {p + 1} done...")
        i += 2

population = population.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="value")
population.rename(columns={"Country Name":"name"},inplace = True)
populationUpdated = population

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
import matplotlib.colors as mc
import colorsys
from random import randint
import re

frames_list = populationUpdated["Year"].unique().tolist()
for i in range(50):
    frames_list.append(populationUpdated["Year"].iloc[-1])

def transform_color(color, amount = 0.5):

    try:
        c = mc.cnames[color]
    except:
        c = color
        c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])


all_names =  populationUpdated["name"].unique().tolist()
random_hex_colors = []
for i in range(len(all_names)):
    random_hex_colors.append('#' + '%06X' % randint(0, 0xFFFFFF))

rgb_colors = [transform_color(i, 1) for i in random_hex_colors]
rgb_colors_opacity = [rgb_colors[x] + (0.825,) for x in range(len(rgb_colors))]
rgb_colors_dark = [transform_color(i, 1.12) for i in random_hex_colors]

fig, ax = plt.subplots(figsize = (36, 20))

num_of_elements = 4

def draw_barchart(Time):

    df_frame = populationUpdated[populationUpdated['Year'].eq(Time)].sort_values(by = 'value', ascending = True).tail(num_of_elements)
    ax.clear()

    normal_colors = dict(zip(populationUpdated['name'].unique(), rgb_colors_opacity))
    dark_colors = dict(zip(populationUpdated['name'].unique(), rgb_colors_dark))
    
    ax.barh(df_frame['name'], df_frame['value'], color = [normal_colors[x] for x in df_frame['name']], height = 0.8,
            edgecolor =([dark_colors[x] for x in df_frame['name']]), linewidth = '6')

    dx = float(df_frame['value'].max()) / 200

    for i, (value, name) in enumerate(zip(df_frame['value'], df_frame['name'])):
        ax.text(value + dx, i + (num_of_elements / 50), '    ' + name,
        size = 36, weight = 'bold', ha = 'left', va = 'center', fontdict = {'fontname': 'Trebuchet MS'})
        ax.text(value + dx, i - (num_of_elements / 50), f'    {value:,.0f}', size = 36, ha = 'left', va = 'center')

    time_unit_displayed = re.sub(r'\^(.*)', r'', str(Time))
    ax.text(1.0, 0.2, time_unit_displayed, transform = ax.transAxes, color = '#666666',
            size = 90, ha = 'right', weight = 'light', fontdict = {'fontname': 'Trebuchet MS'})
    ax.text(1.0, 0.1, "Total: "+ format(int(df_frame['value'].sum()), ',d'), transform = ax.transAxes, color = '#666666',
            size = 62, ha = 'right', weight = 'light', fontdict = {'fontname': 'Trebuchet MS'})
    ax.text(-0.005, 1.14, 'Population per income class from 1960 to 2018', transform = ax.transAxes,
            size = 62, weight = 'bold', ha = 'left', fontdict = {'fontname': 'Trebuchet MS'})

    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis = 'x', colors = '#666666', labelsize = 28)
    ax.set_yticks([])
    ax.set_axisbelow(True)
    ax.margins(0, 0.01)
    ax.grid(which = 'major', axis = 'x', linestyle = '-')

    plt.locator_params(axis = 'x', nbins = 4)
    plt.box(False)
    plt.subplots_adjust(left = 0.075, right = 0.75, top = 0.825, bottom = 0.05, wspace = 0.2, hspace = 0.2)

animator = animation.FuncAnimation(fig, draw_barchart, frames = frames_list)
animator.save("Racing Bar Chart.gif", fps = 25, bitrate = 1800)